In [ ]:
"""
Classify a tweet as being one of the following categories:
601:
602:
603:
604:
605:
606:
607:

Currently only supports english tweets.
"""

In [4]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("..", "..", "..", "src")))

import pandas as pd
from tqdm import tqdm

from common.database import Database
from common.app import App
from common.helpers import Helpers

In [11]:
app_run = App(debug=False)
data_path = os.path.join(app_run.root_dir, "interactive", "data")
df = pd.read_pickle(os.path.join(data_path, "db_en_detect.pkl"))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238523 entries, 0 to 238522
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tweet_id         238523 non-null  object 
 1   covid_theme      238523 non-null  int64  
 2   created_at       238523 non-null  object 
 3   handle           238523 non-null  object 
 4   name             238523 non-null  object 
 5   old_text         152734 non-null  object 
 6   text             237543 non-null  object 
 7   url              238256 non-null  object 
 8   type             238523 non-null  object 
 9   retweets         233824 non-null  float64
 10  favorites        233824 non-null  float64
 11  topic            85303 non-null   float64
 12  subcat           15813 non-null   float64
 13  position         15813 non-null   float64
 14  frame            15813 non-null   float64
 15  theme_hardcoded  32524 non-null   object 
 16  en_detect        238523 non-null  bool

In [22]:
# Keep english tweets that are coded and not 608
df = df[(df["en_detect"] == True) & (~df["topic"].isnull()) & (~df["topic"].isin([608, 608.0]))]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59242 entries, 1225 to 234043
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tweet_id         59242 non-null  object 
 1   covid_theme      59242 non-null  int64  
 2   created_at       59242 non-null  object 
 3   handle           59242 non-null  object 
 4   name             59242 non-null  object 
 5   old_text         32926 non-null  object 
 6   text             59242 non-null  object 
 7   url              59242 non-null  object 
 8   type             59242 non-null  object 
 9   retweets         55742 non-null  float64
 10  favorites        55742 non-null  float64
 11  topic            59242 non-null  float64
 12  subcat           9745 non-null   float64
 13  position         9745 non-null   float64
 14  frame            9745 non-null   float64
 15  theme_hardcoded  1849 non-null   object 
 16  en_detect        59242 non-null  bool   
dtypes: bool(

In [23]:
# Let's work